# Daily Demand over week:

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['font.size'] = 14
import pgeocode
plt.style.use('ggplot')
sns.set(font_scale=1.5)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import scipy.stats as stats
from pygeocoder import Geocoder
from pandas.api.types import CategoricalDtype
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from googletrans import Translator

from sklearn.preprocessing import StandardScaler

In [2]:
austin = pd.read_pickle('../pickled_data/austin_df.pickled')
sanfran = pd.read_pickle('../pickled_data/sf_all_df.pickled')
LA = pd.read_pickle('../pickled_data/LA_new_df.pickled')
chicago = pd.read_pickle('../pickled_data/chicago_trips_df.pickled')
london = pd.read_pickle('../pickled_data/london_counts_df.pickled')
DC = pd.read_pickle('../pickled_data/DC_df.pickled')
porto = pd.read_pickle('../pickled_data/porto_df.pickled')

In [3]:
print(austin.shape)
print(sanfran.shape)
print(LA.shape)
print(chicago.shape)
print(london.shape)
print(DC.shape)
print(porto.shape)

(646431, 10)
(669955, 35)
(755176, 9)
(3603082, 11)
(17414, 12)
(10886, 14)
(17379, 20)


# =====================================

In [4]:
austin.columns

Index(['bikeid', 'checkout_time', 'duration_minutes', 'end_station_name',
       'start_station_name', 'start_time', 'subscriber_type', 'trip_id',
       'Start_Day', 'Month'],
      dtype='object')

In [5]:
austin_cp = austin.copy()
del austin_cp['checkout_time']
del austin_cp['end_station_name']
del austin_cp['start_station_name']

In [6]:
austin_cp.rename(columns={'duration_minutes': 'duration','start_time':'datetime' ,'Start_Day': 'Day'}, inplace=True)

In [7]:
austin_cp['City']='Austin'

In [8]:
austin_cp.head()

,bikeid,duration,datetime,subscriber_type,trip_id,Day,Month,City
0,8.0,41,2015-03-19 19:12:00,Walk Up,9900082882,Thursday,March,Austin
1,141.0,6,2016-10-30 02:06:04,Local365,12617682,Sunday,October,Austin
2,578.0,13,2016-03-11 16:28:27,Local365,9075366,Friday,March,Austin
3,555.0,80,2014-11-23 15:12:00,24-Hour Kiosk (Austin B-cycle),9900319298,Sunday,November,Austin
4,86.0,25,2017-04-16 15:39:13,Walk Up,14468597,Sunday,April,Austin


In [9]:
austin_cp.datetime.dtypes

dtype('<M8[ns]')

In [10]:
austin_cp['date'] = austin_cp.datetime.dt.date

In [11]:
austin_cp.groupby(['date']).count()

,bikeid,duration,datetime,subscriber_type,trip_id,Day,Month,City
date,,,,,,,,
2013-12-21,103,103,103,103,103,103,103,103
2013-12-22,117,117,117,117,117,117,117,117
2013-12-23,96,96,96,96,96,96,96,96
2013-12-24,85,85,85,85,85,85,85,85
2013-12-25,145,145,145,145,145,145,145,145
...,...,...,...,...,...,...,...,...
2017-07-27,370,370,370,370,370,370,370,370
2017-07-28,402,402,402,402,402,402,402,402
2017-07-29,425,425,425,425,425,425,425,425


# =====================================

In [12]:
sanfran.columns

Index(['id', 'duration', 'start_date', 'start_station_name',
       'start_station_id', 'end_date', 'end_station_name', 'end_station_id',
       'bike_id', 'subscription_type', 'date', 'Day', 'End_Day', 'Month',
       'max_temperature_f', 'mean_temperature_f', 'min_temperature_f',
       'max_dew_point_f', 'mean_dew_point_f', 'min_dew_point_f',
       'max_humidity', 'mean_humidity', 'min_humidity',
       'max_sea_level_pressure_inches', 'mean_sea_level_pressure_inches',
       'min_sea_level_pressure_inches', 'max_visibility_miles',
       'mean_visibility_miles', 'min_visibility_miles', 'max_wind_Speed_mph',
       'mean_wind_speed_mph', 'cloud_cover', 'wind_dir_degrees', 'zip_code',
       'mean_humidity_binary'],
      dtype='object')

In [13]:
sanfran_cp = sanfran.copy()
del sanfran_cp['id']
del sanfran_cp['start_station_name']
del sanfran_cp['start_station_id']
del sanfran_cp['end_date']
del sanfran_cp['end_station_name']
del sanfran_cp['end_station_id']
del sanfran_cp['End_Day']
del sanfran_cp['max_temperature_f']
del sanfran_cp['min_temperature_f']
del sanfran_cp['max_dew_point_f']
del sanfran_cp['min_dew_point_f']
del sanfran_cp['mean_dew_point_f']
del sanfran_cp['max_humidity']
del sanfran_cp['min_humidity']
del sanfran_cp['max_sea_level_pressure_inches']
del sanfran_cp['mean_sea_level_pressure_inches']
del sanfran_cp['min_sea_level_pressure_inches']
del sanfran_cp['mean_visibility_miles']
del sanfran_cp['min_visibility_miles']
del sanfran_cp['max_visibility_miles']
del sanfran_cp['max_wind_Speed_mph']
del sanfran_cp['mean_wind_speed_mph']
del sanfran_cp['wind_dir_degrees']
del sanfran_cp['cloud_cover']
del sanfran_cp['zip_code']

In [14]:
sanfran_cp.rename(columns={'start_time':'datetime' ,'mean_humidity': 'humidity','mean_humidity_binary':'humidity_binary'}, inplace=True)

In [15]:
sanfran_cp['City']='San Francisco'

In [16]:
sanfran_cp.head()

,duration,start_date,bike_id,subscription_type,date,Day,Month,mean_temperature_f,humidity,humidity_binary,City
0,63,2013-08-29 14:13:00,520,Subscriber,8/29/2013,Thursday,August,70.4,73.2,1.0,San Francisco
1,70,2013-08-29 14:42:00,661,Subscriber,8/29/2013,Thursday,August,70.4,73.2,1.0,San Francisco
2,71,2013-08-29 10:16:00,48,Subscriber,8/29/2013,Thursday,August,70.4,73.2,1.0,San Francisco
3,77,2013-08-29 11:29:00,26,Subscriber,8/29/2013,Thursday,August,70.4,73.2,1.0,San Francisco
4,83,2013-08-29 12:02:00,319,Subscriber,8/29/2013,Thursday,August,70.4,73.2,1.0,San Francisco


#  =====================================

In [17]:
LA.columns

Index(['bike_id', 'duration', 'end_time', 'passholder_type', 'start_time',
       'trip_id', 'trip_route_category', 'Day', 'Month'],
      dtype='object')

In [18]:
LA_cp = LA.copy()
del LA_cp['end_time']
del LA_cp['trip_id']
del LA_cp['trip_route_category']

In [19]:
LA_cp.rename(columns={'start_time':'datetime'}, inplace=True)

In [20]:
LA_cp['City']='LA'

In [21]:
LA.shape

(755176, 9)

#  =====================================

In [22]:
chicago.columns

Index(['trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'Day'],
      dtype='object')

#  =====================================

In [23]:
london.columns

Index(['timestamp', 'cnt', 't1', 't2', 'hum', 'wind_speed', 'weather_code',
       'is_holiday', 'is_weekend', 'season', 'Day', 'hum_binary'],
      dtype='object')

#  =====================================

In [24]:
DC.shape

(10886, 14)

In [25]:
DC.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count',
       'hum_binary', 'Day'],
      dtype='object')

#  =====================================

In [26]:
porto.shape

(17379, 20)

In [27]:
porto.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt', 'Day', 'Month', 'hum_binary'],
      dtype='object')

In [28]:
del porto['instant']

In [29]:
porto.rename(columns={'dteday': 'datetime','hum':'humidity' ,'cnt': 'count'}, inplace=True)

In [30]:
porto.head()

,datetime,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,humidity,windspeed,casual,registered,count,Day,Month,hum_binary
0,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16,Saturday,January,1
1,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40,Saturday,January,1
2,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32,Saturday,January,1
3,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13,Saturday,January,1
4,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1,Saturday,January,1


In [31]:
austin.shape

(646431, 10)

In [32]:
austin.start_time.dt.date.sort_values().unique()[-10:]

array([datetime.date(2017, 7, 22), datetime.date(2017, 7, 23),
       datetime.date(2017, 7, 24), datetime.date(2017, 7, 25),
       datetime.date(2017, 7, 26), datetime.date(2017, 7, 27),
       datetime.date(2017, 7, 28), datetime.date(2017, 7, 29),
       datetime.date(2017, 7, 30), datetime.date(2017, 7, 31)],
      dtype=object)

In [33]:
aust_weather = pd.read_csv('/Users/vilvaki/Desktop/GA/bikes/austin-bike/austin_weather.csv')

In [39]:
aust_weather.shape

(1319, 22)

In [57]:
aust_weather['datetime'] = pd.to_datetime(aust_weather['Date']).dt.date
# aust_weather['datetime'].dt.date.sort_values().unique()[-10:]

In [58]:
austin['start_date'] = austin.start_time.dt.date
aust_weather['datetime'][0]

datetime.date(2013, 12, 21)

In [59]:
AUSTIN = pd.merge(austin, aust_weather, left_on="start_date", right_on="datetime", how="left", sort=False)


In [60]:
AUSTIN.shape

(646431, 33)

In [61]:
AUSTIN.isnull().sum()

bikeid                        0
checkout_time                 0
duration_minutes              0
end_station_name              0
start_station_name            0
start_time                    0
subscriber_type               0
trip_id                       0
Start_Day                     0
Month                         0
start_date                    0
Date                          0
TempHighF                     0
TempAvgF                      0
TempLowF                      0
DewPointHighF                 0
DewPointAvgF                  0
DewPointLowF                  0
HumidityHighPercent           0
HumidityAvgPercent            0
HumidityLowPercent            0
SeaLevelPressureHighInches    0
SeaLevelPressureAvgInches     0
SeaLevelPressureLowInches     0
VisibilityHighMiles           0
VisibilityAvgMiles            0
VisibilityLowMiles            0
WindHighMPH                   0
WindAvgMPH                    0
WindGustMPH                   0
PrecipitationSumInches        0
Events  

In [62]:
AUSTIN.head()

,bikeid,checkout_time,duration_minutes,end_station_name,start_station_name,start_time,subscriber_type,trip_id,Start_Day,Month,...,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events,datetime
0,8.0,19:12:00,41,Trinity & 6th Street,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,9900082882,Thursday,March,...,29.88,10,8,2,12,5,17,0,,2015-03-19
1,141.0,2:06:04,6,South Congress & Academy,2nd & Congress,2016-10-30 02:06:04,Local365,12617682,Sunday,October,...,30.07,10,10,8,10,2,14,0,,2016-10-30
2,578.0,16:28:27,13,Convention Center / 4th St. @ MetroRail,Bullock Museum @ Congress & MLK,2016-03-11 16:28:27,Local365,9075366,Friday,March,...,29.87,10,7,2,9,3,15,0.36,"Rain , Thunderstorm",2016-03-11
3,555.0,15:12:00,80,Toomey Rd @ South Lamar,Capitol Station / Congress & 11th,2014-11-23 15:12:00,24-Hour Kiosk (Austin B-cycle),9900319298,Sunday,November,...,29.51,10,10,10,16,5,27,0,,2014-11-23
4,86.0,15:39:13,25,MoPac Pedestrian Bridge @ Veterans Drive,Rainey St @ Cummings,2017-04-16 15:39:13,Walk Up,14468597,Sunday,April,...,29.95,10,10,10,14,6,21,0,,2017-04-16
